###### Transfer Learning

In [1]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets,transforms,models
import matplotlib.pyplot as plt
%matplotlib inline
%config inlineBackend.figure_format='retina'

In [2]:
data_dir='Cat_Dog_data'
#Define transforms for training data
train_transforms=transforms.Compose([transforms.RandomRotation(30),
                                    transforms.RandomResizedCrop(224),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406],
                                                        [0.229, 0.224, 0.225])])
#Define transforms for testing data
test_transforms=transforms.Compose([transforms.Resize(255),
                                   transforms.CenterCrop(224),
                                   transforms.ToTensor(),
                                   transforms.Normalize([0.485, 0.456, 0.406],
                                                        [0.229, 0.224, 0.225])])

#Pass transforms
train_data=datasets.ImageFolder(data_dir+'/train',transform=train_transforms)
test_data=datasets.ImageFolder(data_dir+'/test',transform=test_transforms)
trainloader=torch.utils.data.DataLoader(train_data,batch_size=64,shuffle=True)
testloader=torch.utils.data.DataLoader(test_data,batch_size=64)

In [3]:
model=models.densenet121(pretrained=True)
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [4]:
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad = False
from collections import OrderedDict
classifier=nn.Sequential(OrderedDict([
                          ('fc1', nn.Linear(1024, 500)),
                          ('relu', nn.ReLU()),
                          ('fc2', nn.Linear(500, 2)),
                          ('output', nn.LogSoftmax(dim=1))
                          ]))
model.classifier=classifier

In [5]:
import time

In [6]:
for device in ['cpu', 'cuda']:
    criterion=nn.NLLLoss()
    # Only train the classifier parameters, feature parameters are frozen
    optimizer=optim.Adam(model.classifier.parameters(),lr=0.001)
    model.to(device)
    for ii,(inputs, labels) in enumerate(trainloader):
        # Move input and label tensors to the GPU
        inputs,labels=inputs.to(device),labels.to(device)
        start=time.time()
        outputs=model.forward(inputs)
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        if ii==3:
            break
    print(f"Device = {device}; Time per batch: {(time.time() - start)/3:.3f} seconds")

Device = cpu; Time per batch: 11.426 seconds


AssertionError: Torch not compiled with CUDA enabled

>**Exercise:** Train a pretrained models to classify the cat and dog images. Continue with the DenseNet model, or try ResNet, it's also a good model to try out first. Make sure you are only training the classifier and the parameters for the features part are frozen.

In [7]:
# Use GPU if it's available
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=models.densenet121(pretrained=True)
# Freeze parameters so we don't backprop through them
for param in model.parameters():
    param.requires_grad=False    
model.classifier=nn.Sequential(nn.Linear(1024, 256),
                                 nn.ReLU(),
                                 nn.Dropout(0.2),
                                 nn.Linear(256, 2),
                                 nn.LogSoftmax(dim=1))
criterion=nn.NLLLoss()
# Only train the classifier parameters, feature parameters are frozen
optimizer=optim.Adam(model.classifier.parameters(), lr=0.003)
model.to(device);

In [ ]:
epochs=1
steps=0
running_loss=0
print_every=5
for epoch in range(epochs):
    for inputs,labels in trainloader:
        steps+= 1
        # Move input and label tensors to the default device
        inputs,labels=inputs.to(device),labels.to(device)
        optimizer.zero_grad()
        logps=model.forward(inputs)
        loss=criterion(logps, labels)
        loss.backward()
        optimizer.step()
        running_loss+=loss.item()       
        if steps % print_every==0:
            test_loss=0
            accuracy=0
            model.eval()
            with torch.no_grad():
                for inputs,labels in testloader:
                    inputs,labels=inputs.to(device),labels.to(device)
                    logps=model.forward(inputs)
                    batch_loss=criterion(logps,labels)       
                    test_loss+=batch_loss.item()                    
                    # Calculate accuracy
                    ps=torch.exp(logps)
                    top_p,top_class=ps.topk(1,dim=1)
                    equals=top_class==labels.view(*top_class.shape)
                    accuracy+=torch.mean(equals.type(torch.FloatTensor)).item()                   
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()

Epoch 1/1.. Train loss: 0.739.. Test loss: 0.269.. Test accuracy: 0.957
Epoch 1/1.. Train loss: 0.393.. Test loss: 0.179.. Test accuracy: 0.947
Epoch 1/1.. Train loss: 0.281.. Test loss: 0.092.. Test accuracy: 0.975
Epoch 1/1.. Train loss: 0.259.. Test loss: 0.078.. Test accuracy: 0.975
Epoch 1/1.. Train loss: 0.285.. Test loss: 0.091.. Test accuracy: 0.967
Epoch 1/1.. Train loss: 0.222.. Test loss: 0.085.. Test accuracy: 0.972
Epoch 1/1.. Train loss: 0.213.. Test loss: 0.080.. Test accuracy: 0.970
Epoch 1/1.. Train loss: 0.156.. Test loss: 0.062.. Test accuracy: 0.980
Epoch 1/1.. Train loss: 0.158.. Test loss: 0.064.. Test accuracy: 0.977
Epoch 1/1.. Train loss: 0.187.. Test loss: 0.052.. Test accuracy: 0.981
Epoch 1/1.. Train loss: 0.219.. Test loss: 0.057.. Test accuracy: 0.980
